In [ ]:
!cp -r /kaggle/input/rj01204891/myProject /kaggle/working/galtransl_project

!git clone https://github.com/SakuraLLM/Sakura-13B-Galgame.git

!git clone https://github.com/xd2333/GalTransl.git
# !git clone --no-checkout https://github.com/xd2333/GalTransl.git
# %cd /kaggle/working/GalTransl
# !git checkout f43a369
%cd /kaggle/working/GalTransl

!pip install virtualenv
!virtualenv env
!/kaggle/working/GalTransl/env/bin/pip install -r requirements.txt
!/kaggle/working/GalTransl/env/bin/pip install opencc==1.1.6

%cd /kaggle/working/Sakura-13B-Galgame

!pip install "diskcache>=5.6.1"
!pip install llama-cpp-python -i https://sakurallm.github.io/llama-cpp-python/whl/cu121
!pip install -q -r requirements.llamacpp.txt
!pip install -q pyngrok


In [ ]:
#REPO = "SakuraLLM/Sakura-14B-Qwen2beta-v0.9-GGUF"
#MODEL = "sakura-14b-qwen2beta-v0.9-iq4_xs_ver2"
#REPO = "SakuraLLM/Sakura-14B-LNovel-v0.9b-GGUF"
#MODEL = "sakura-13b-lnovel-v0.9b-Q4_K_M"

#REPO = "SakuraLLM/Sakura-14B-Qwen2beta-v0.9.2-GGUF"
#MODEL = "sakura-14b-qwen2beta-v0.9.2-iq4xs"
#MODEL = "sakura-14b-qwen2beta-v0.9.2-q6k"

REPO="SakuraLLM/GalTransl-7B-v2"
MODEL="Galtransl-7B-v2-Q6_K"


ngrokToken = ""

# 启动sakura模型
%cd /kaggle/working/Sakura-13B-Galgame
from huggingface_hub import hf_hub_download
from pathlib import Path
MODEL_PATH = f"./models/{MODEL}.gguf"
if not Path(MODEL_PATH).exists():
    hf_hub_download(repo_id=REPO, filename=f"{MODEL}.gguf", local_dir="models/")

import subprocess
import threading
import time

def local_model(dic_status):
    cmd = f"python server.py --model_name_or_path {MODEL_PATH} --llama_cpp --use_gpu --model_version 0.9 --trust_remote_code --no-auth"
    
    p = subprocess.Popen(cmd, shell=True, stdout=subprocess.DEVNULL, stderr=subprocess.PIPE)
    for line in p.stderr:
        if not dic_status['status']:
            s2 = line.decode()
            if "INFO Running on http://127.0.0.1:5000" in s2:
                dic_status['status'] = True
            dic_status['msg'] = s2
        #print(line.decode(), end='')
dic_status = {'status': False, 'msg': ''}

threading.Thread(target=local_model, daemon=True, args=(dic_status,)).start()

s = ""
while not dic_status['status']:
    if s != dic_status['msg']:
        s = dic_status['msg']
        print(s)
print("模型启动成功")


# ngrok：本地HTTP服务用于下载
PORT = 8000
DIRECTORY = "/kaggle/working"
def local_http():
    import http.server
    import socketserver

    Handler = http.server.SimpleHTTPRequestHandler
    Handler.directory = DIRECTORY

    with socketserver.TCPServer(("", PORT), Handler) as httpd:
        print("HTTP server is running at port", PORT)
        print("Server directory is", DIRECTORY)
        httpd.serve_forever()

if ngrokToken:
    threading.Thread(target=local_http, daemon=True, args=()).start()
        
    from pyngrok import conf, ngrok
    conf.get_default().auth_token = ngrokToken
    conf.get_default().monitor_thread = False
    ssh_tunnels = ngrok.get_tunnels(conf.get_default())
    if len(ssh_tunnels) == 0:
        ssh_tunnel = ngrok.connect(PORT)
        print('address：'+ssh_tunnel.public_url)
    else:
        print('address：'+ssh_tunnels[0].public_url)


try:
    result = subprocess.run(
        ["/kaggle/working/GalTransl/env/bin/python", 
        "-m", "GalTransl",
        "-p", "/kaggle/working/galtransl_project",
        "-t" "galtransl-v2"],
        cwd="/kaggle/working/GalTransl",
        stdout=subprocess.DEVNULL,  # 屏蔽标准输出
        stderr=subprocess.DEVNULL,  # 屏蔽错误输出
        timeout=int(11.5*3600)
    )
    print("Process completed successfully.")
except subprocess.TimeoutExpired:
    print("Process timed out and was terminated.")
    
# %cd /kaggle/working/GalTransl
# !/kaggle/working/GalTransl/env/bin/python -m GalTransl -p /kaggle/working/galtransl_project -t galtransl-v2

import shutil
shutil.make_archive('/kaggle/working/result', 'zip', '/kaggle/working/galtransl_project')